In [1]:
from torch.utils.data import Dataset
import os
import numpy as np
from scipy import fftpack as fft
import cv2
import pandas as pd

classes = ["fall", "jump", "not_fall", "run", "run_sit", "sit_run", "walk"]

overlap = 128
n_rd_history = 256
overlap_count = 0
min_iq = 0
max_iq = 0.05
min_rf = 0
max_rf = 45


class CSVDopplerDatagen(Dataset):
    def __init__(self,
                 csv='/mnt/c/Users/Admin/Downloads/work/uwb/uwb-pose-prediction/annotations.csv',
                 src='/mnt/c/Users/Admin/Downloads/work/uwb/uwb-pose-prediction/train/train'):
        self.csv = pd.read_csv(csv).fillna(-1)
        self.src = src

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):
        row = self.csv.iloc[idx]
        iq = np.load(os.path.join(
            self.src,
            row['id'] + '.npy'
        ))
        if iq.shape[0] > 2560:
            iq = iq[-2560:]
        range_frequency = self.range_frequency([iq])[0]
        image = np.stack((
            iq / max_iq,
            range_frequency / max_rf,
            np.zeros_like(iq),
        ))
        return image, row['class']

    @staticmethod
    def range_frequency(datas):
        Range_frequency_frame = []
        for data in datas:
            jitter = 1e-10
            noise_threshold = -45
            dB = True
            rd_history = np.hanning(data.shape[0])[:, None] * np.array(data)
            # Range-Doppler
            rd = fft.fft(rd_history, axis=0)
            rd = fft.fftshift(rd, axes=0)
            rd = abs(rd)
            if dB:
                rd = 20 * np.log10(rd + jitter)
                rd[rd < noise_threshold] = noise_threshold
            Range_frequency_frame.append(rd)
        return np.stack(Range_frequency_frame)



if __name__ == '__main__':
    dset = CSVDopplerDatagen()
    min_rf = float('inf')
    min_iq = float('inf')
    max_rf = -float('inf')
    max_iq = -float('inf')
    lengths = {2560: 0, 7680: 0}
    for i in range(len(dset)):
        data, _ = dset[i]
        # min_iq = min(min_iq, np.abs(data[0]).min())
        # max_iq = max(max_iq, np.abs(data[0]).max())
        # min_rf = min(min_rf, np.abs(data[1]).min())
        # max_rf = max(max_rf, np.abs(data[1]).max())
        lengths[data.shape[1]] += 1
    x = dset[0]


In [2]:
from torch import nn
from timm.models import convnext

class ViTCls(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = convnext.convnext_base(pretrained=True)
        self.cls = nn.Linear(self.encoder.num_features, 7)

    def forward(self, images):
        x = self.encoder.forward_features(images)
        x = self.encoder.forward_head(x, pre_logits=True)
        y = self.cls(x)
        return y



In [11]:
from models import ViTCls
from datagen import CSVDopplerDatagen
from torch.utils.data import DataLoader
from torch import nn
import torch
from timm.scheduler.cosine_lr import CosineLRScheduler
import tensorflow as tf
from sklearn.model_selection import KFold
import pandas as pd
from sklearn.metrics import classification_report
import json
import warnings
warnings.filterwarnings("ignore")


if __name__ == '__main__':
    n_epochs = 15
    batch_size = 8
    n_folds = 5
    device = 'cuda'
    model = ViTCls().to(device)
    print(0)
    data = {'fold': [], 'epoch': [], 'acc': [], 'mse': [], 'cce': []}

    train_dataset = CSVDopplerDatagen()
    val_dataset = CSVDopplerDatagen(csv='/mnt/c/Users/Admin/Downloads/work/uwb/uwb-pose-prediction/sample_submission.csv',
                                    src='/mnt/c/Users/Admin/Downloads/work/uwb/uwb-pose-prediction/test/test')
    val_dataset.csv = val_dataset.csv.fillna(-1)
    train_loader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              shuffle=True,
                              num_workers=4
                              )
    val_loader = DataLoader(val_dataset,
                            batch_size=1,
                            shuffle=False,
                            num_workers=4
                            )

    criterion2 = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
    schedule = CosineLRScheduler(optimizer,
                                 t_initial=n_epochs,
                                 cycle_mul=1,
                                 lr_min=1e-6,
                                 cycle_decay=0.1,
                                 warmup_lr_init=1e-7,
                                 warmup_t=10,
                                 cycle_limit=1,
                                 t_in_epochs=True,
                                 noise_range_t=None,
                                 )
    max_acc = 0
    for epoch in range(n_epochs):
        print('Epoch:', epoch + 1)
        progbar = tf.keras.utils.Progbar(len(train_loader))
        model.train()
        for idx, (x, cls) in enumerate(train_loader):
            # data_ft = data_ft.to(device)
            # data_dt = data_dt.to(device)
            x = x.to(device).float()
            cls = cls.to(device)
            # output = model(data_ft, data_dt)
            logit = model(
                x  # [:, 0]
            )
            loss = criterion2(logit, cls)
            optimizer.zero_grad()
            loss.backward()
            _, predicted = logit.max(1)
            acc = predicted.eq(cls).sum().item() / cls.size(0)
            optimizer.step()
            printlog = [
                # ('loss1', loss1.cpu().detach().numpy()),
                ('loss2', loss.cpu().detach().numpy()),
                ('acc', acc)
            ]
            progbar.update(idx + 1, printlog)
        schedule.step(epoch + 1)
        model.eval()
        progbar = tf.keras.utils.Progbar(len(val_loader))
        predict = []
        with torch.no_grad():
            for idx, (x, cls) in enumerate(val_loader):
                if cls[0] > -1:
                    predict.extend(cls.cpu().detach().numpy().tolist())
                    continue
                x = x.to(device).float()
                cls = cls.to(device)
                logit = model(
                    x  # [:, 0]
                )
                _, predicted = logit.max(1)
                predict.extend(predicted.cpu().detach().numpy().tolist())
        df = val_dataset.csv.copy()
        df['class'] = predict
        df.to_csv(f'logs/json_files/{epoch:02d}.csv', index=False)

0
Epoch: 1
81/81 [==============================] - 38s 468ms/step - loss2: 2.0390 - acc: 0.1704
Epoch: 2
81/81 [==============================] - 39s 474ms/step - loss2: 1.9260 - acc: 0.1698
Epoch: 3
81/81 [==============================] - 39s 481ms/step - loss2: 1.6742 - acc: 0.3713
Epoch: 4
81/81 [==============================] - 40s 489ms/step - loss2: 1.3947 - acc: 0.5161
Epoch: 5
81/81 [==============================] - 39s 483ms/step - loss2: 1.1753 - acc: 0.5642
Epoch: 6
81/81 [==============================] - 40s 486ms/step - loss2: 1.0139 - acc: 0.6182
Epoch: 7
81/81 [==============================] - 40s 486ms/step - loss2: 0.8510 - acc: 0.7352
Epoch: 8
81/81 [==============================] - 40s 488ms/step - loss2: 0.6905 - acc: 0.8007
Epoch: 9
81/81 [==============================] - 40s 485ms/step - loss2: 0.5120 - acc: 0.8408
Epoch: 10
81/81 [==============================] - 40s 486ms/step - loss2: 0.4466 - acc: 0.8655
Epoch: 11
81/81 [==============================